In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-plot 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import Normalizer, StandardScaler
from scipy.io import arff
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dense, Dropout, Activation, Embedding
from sklearn.preprocessing import label_binarize
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import  roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
from sklearn.model_selection import KFold,cross_val_score
from scipy import interp
from itertools import cycle
kfold=KFold(5,True,3)
cvscores = []

In [ ]:
# Read the Dataset and Rename the Column
df = pd.read_csv("/content/drive/MyDrive/Code/dataset/Preprocessed_data.csv",header=0,names=columns_to_named)
y = df.iloc[:,-1].values
X = df.iloc[:,1:].values
from sklearn.preprocessing import label_binarize
y = label_binarize(y, classes=[0, 1, 2, 3, 4, 5])
n_classes = y.shape[1]
print(n_classes)

6


In [ ]:
sc = StandardScaler()
for train, test in kfold.split(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=101)
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    model = Sequential()
    model.add(Dense(800, input_dim=13, activation='relu'))
    model.add(Dropout(0.9))
    model.add(Dense(400, activation='relu'))
    model.add(Dropout(0.9))
    model.add(Dense(6,activation='softmax'))

    # compile model
    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy','mae'])
    history = model.fit(X_train, y_train, epochs = 10, batch_size=64, verbose=1,shuffle=False)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/10
5968/5968 [==============================] - 43s 7ms/step - loss: 0.1096 - accuracy: 0.9740 - mae: 0.0144
Epoch 2/10
5968/5968 [==============================] - 43s 7ms/step - loss: 0.0561 - accuracy: 0.9958 - mae: 0.0026
Epoch 3/10
5968/5968 [==============================] - 43s 7ms/step - loss: 0.0203 - accuracy: 0.9967 - mae: 0.0019
Epoch 4/10
5968/5968 [==============================] - 43s 7ms/step - loss: 0.0194 - accuracy: 0.9969 - mae: 0.0017
Epoch 5/10
5968/5968 [==============================] - 43s 7ms/step - loss: 0.0270 - accuracy: 0.9969 - mae: 0.0017
Epoch 6/10
5968/5968 [==============================] - 44s 7ms/step - loss: 0.0220 - accuracy: 0.9971 - mae: 0.0016
Epoch 7/10
5968/5968 [==============================] - 45s 8ms/step - loss: 0.1321 - accuracy: 0.9972 - mae: 0.0015
Epoch 8/10
5968/5968 [==============================] - 44s 7ms/step - loss: 0.0408 - accuracy: 0.9974 - mae: 0.0013
Epoch 9/10
5968/5968 [==============================] - 44s 7ms/

In [ ]:
target_names = ['Normal', 'WrongSetup', 'DDoS', 'Probing', 'Scan', 'MITM']

In [ ]:
#test accuracy
expected_test = y_test
predicted_test = model.predict(X_test)
print("Test_Accuracy:",metrics.accuracy_score(expected_test, np.round(predicted_test)))
print("Test_Precision:",metrics.precision_score(expected_test, np.round(predicted_test), average='weighted'))
print("Test_Recall:",metrics.recall_score(expected_test, np.round(predicted_test), average='weighted'))
print("Test_f1-score:",metrics.f1_score(expected_test, np.round(predicted_test), average='weighted'))
data = classification_report(expected_test,np.round(predicted_test),target_names=target_names)
print(data)

#train accuracy
expected_train = y_train
predicted_train = model.predict(X_train)
print("Train_Accuracy:",metrics.accuracy_score(expected_train, np.round(predicted_train)))
print("Train_Precision:",metrics.precision_score(expected_train, np.round(predicted_train), average='weighted'))
print("Train_Recall:",metrics.recall_score(expected_train, np.round(predicted_train), average='weighted'))
print("Train_f1-score:",metrics.f1_score(expected_train, np.round(predicted_train), average='weighted'))
data = classification_report(expected_train,np.round(predicted_train),target_names=target_names)
print(data)


Test_Accuracy: 0.9990993444065098
Test_Precision: 0.9991025037769024
Test_Recall: 0.9990993444065098
Test_f1-score: 0.9990989125518646
              precision    recall  f1-score   support

      Normal       1.00      0.99      1.00     15873
  WrongSetup       1.00      1.00      1.00     16377
        DDoS       1.00      1.00      1.00     15790
     Probing       1.00      1.00      1.00     15819
        Scan       1.00      1.00      1.00     15728
        MITM       1.00      1.00      1.00     15899

   micro avg       1.00      1.00      1.00     95486
   macro avg       1.00      1.00      1.00     95486
weighted avg       1.00      1.00      1.00     95486
 samples avg       1.00      1.00      1.00     95486

Train_Accuracy: 0.9989972246949783
Train_Precision: 0.9990036564525343
Train_Recall: 0.9989972246949783
Train_f1-score: 0.9989979644595448
              precision    recall  f1-score   support

      Normal       1.00      0.99      1.00     63162
  WrongSetup       1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
